In [5]:
%%writefile app.py
from flask import Flask, Response, render_template, make_response, jsonify, request as req
from flask_cors import CORS
from boto3.dynamodb.conditions import Key, Attr
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import psycopg2
import boto3
import json
import re
import os

load_dotenv('.env')
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

dynamodb = boto3.resource('dynamodb', region_name='ap-southeast-2',
  aws_access_key_id=AWS_ACCESS_KEY_ID,
  aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

table_prices = dynamodb.Table('IngredientPrices')

conn = psycopg2.connect(
    "dbname=eltrial user=top password=1234"
)

app = Flask(__name__)
CORS(app)

includes = re.compile("หมู|ไก่|วัว|^ไข่$|ไข่ไก่|^นม")
excludes = re.compile("นมข้นหวาน|ผง|คนอร์|มาม่า|คาร์เนชั่น|หมูกรอบ|กระดูก|ปู|^ซอส|^เครื่องเทศ|^เต้าหู้|อินทผาลัม|ลูกชิ้น|หมูแฮม|ไข่แดงเค็ม|น้ำมัน|คอร์|ไส้กรอก|หัวใจ|กระเพาะ|ปอด|รสดี|คนอ|น้ำซุป|หมูยอ|น้ำ|ซีอิ๊ว|ชุด|^พริกไทย|ซีอิ้ว|สต๊อก|แป้ง|^พริก|เลือด|ชุป|รากผักชี|บะหมี่กึ่งสำเร็จรูป|ข้นหวาน|กากหมู|นมข้น|หมูแดง|แคปหมู|มันหมู|^ซอส|นมถั่วเหลือง|\*\*|^เหล้าจีน|อัลมอนด์|หมูหยอง|สูตร|ซุปก้อน|^เครื่อง|ส่วนผสม|แคบหมู")
excludes_units = re.compile("สำหรับ|ครึ่1/2|ปริมานตามใจชอบ")

ingredient_kws = re.compile('pork_ขาหมู|pork_ซี่โครงหมู|pork_ตับหมู|pork_เนื้อก้อน|\
pork_เนื้อแดง|pork_เนื้อไหล่|pork_สะโพกหมู|pork_สันนอก|pork_สันใน|\
pork_สามชั้น|pork_ไส้ตันหมู|pork_ไส้ใหญ่หมู|pork_ไส้อ่อนหมู|pork_หมูบด|\
beef_ขอบกระด้ง|beef_ขี้ริ้ว|beef_เครื่องใน|beef_ดอกจอก|beef_น่อง|beef_ซี่โครง|\
beef_เนื้อแดง|beef_เนื้อปลีก|beef_เศษเนื้อ|beef_สันคอ|beef_เนื้อวัวบด|beef_เนื้อก้อน|\
beef_สันใน|beef_สันนอก|chicken_ไก่บ้าน|chicken_ไก่ทั้งตัว|chicken_ไก่ผ่าซีก|\
chicken_อกไก่|chicken_ซี่โครงไก่|chicken_ปีกบนไก่|chicken_ปีกกลางไก่|\
chicken_ปีกไก่เต็ม|chicken_น่องไก่|chicken_สะโพกไก่|chicken_เครื่องในไก่|\
chicken_ตีนไก่|chicken_เนื้อไก่บด|chicken_ไก่มีชีวิตหน้าฟาร์ม|eggs_ไข่ไก่เบอร์ 0|\
eggs_ไข่ไก่เบอร์ 1|eggs_ไข่ไก่เบอร์ 2|eggs_ไข่ไก่เบอร์ 3|eggs_ไข่ไก่เบอร์ 4|\
eggs_ไข่ไก่เบอร์ 5|eggs_ไข่ไก่ต้ม|eggs_ไข่ไก่คละหน้าฟาร์ม|นม')

ingredient_translate = {
    "หมูสับ": "pork_เนื้อก้อน",
    "เนื้อหมูบด": "pork_หมูบด",
    "หมูบด": "pork_หมูบด",
    "ไข่ไก่": "eggs_ไข่ไก่เบอร์ 2",
    "ไขไก่": "eggs_ไข่ไก่เบอร์ 2",
    "หมูสามชั้น": "pork_สามชั้น",
    "เนื้อหมู": "pork_เนื้อก้อน",
    "ปีกไก่บน": "chicken_ปีกบนไก่", 
    "นมสด": "นม",
    "สันในไก่": "chicken_อกไก่",
    "อกไก่": "chicken_อกไก่",
    "มันไก่": "chicken_น่องไก่",
    "ไก่สด": "chicken_อกไก่",
    "สะโพกไก่": "chicken_สะโพกไก่",
    "สันคอหมู": "pork_เนื้อก้อน",
    "คอหมู": "pork_เนื้อก้อน",
    "ซี่โครงหมู": "pork_ซี่โครงหมู",
    "ปีกไก่กลาง": "chicken_ปีกกลางไก่",
    "ไก่อก": "chicken_อกไก่",
    "หมูสันนอก": "pork_สันนอก",
    "ตับไก่": "chicken_เครื่องในไก่",
    "เนื้อไก่": "chicken_อกไก่",
    "ปีกไก่": "chicken_ปีกไก่เต็ม",
    "หมูส่วนสะโพก": "pork_สะโพกหมู",
    "หมูสันใน": "pork_สันใน",
    "เนื่อหมู": "pork_เนื้อก้อน",
    "เนื้อสันในวัว": "beef_สันใน",
    "เนื้อวัว": "beef_เนื้อก้อน",
    "โครงไก่": "chicken_ซี่โครงไก่",
    "สันนอกหมู": "pork_สันนอก",
    "น่องไก่": "chicken_น่องไก่",
    "ไก่บด": "chicken_เนื้อไก่บด",
    "ปีกบนไก่": "chicken_ปีกบนไก่",
    "หมูบก": "pork_หมูบด",
    "หมูสามชั้น": "pork_สามชั้น",
    "ใส้หมู": "pork_ไส้อ่อนหมู",
    "ไส้หมู": "pork_ไส้อ่อนหมู",
    "หนังไก่": "chicken_น่องไก่",
    "ไส้ใหญ่หมู": "pork_ไส้ใหญ่หมู",
    "เอ็นข้อไก่": "chicken_ตีนไก่",
    "สันในหมู": "pork_สันใน",
    "ตีนไก่": "chicken_ตีนไก่",
    "ตับหมู": "pork_ตับหมู",
    "ปีกกลางไก่": "chicken_ปีกกลางไก่",
    "ไก่สับ": "chicken_น่องไก่",
    "หนังหมู": "pork_สามชั้น",
}

default_ingredient = {
    "นม": "นม",
    "ไข่": "eggs_ไข่ไก่เบอร์ 2",
    "หมู": "pork_เนื้อก้อน",
    "ไก่": "chicken_น่องไก่",
    "วัว": "beef_เนื้อก้อน",
}

frac_pattern = re.compile("\d+\/\d+")
num_pattern = re.compile("\d+")

ingd_patterns = []
for u in ingredient_translate:
        ingd_patterns.append(u)
        
ingd_pattern = re.compile("|".join(ingd_patterns))

ingd_default_patterns = []
for u in default_ingredient:
        ingd_default_patterns.append(u)
        
ingd_default_pattern = re.compile("|".join(ingd_default_patterns))

units = {
    "กรัม": (0.001, "กก."),
    "g": (0.001, "กก."),
    "grams": (0.001, "กก."),
    "กระม": (0.001, "กก."),
    "กิโล": (1., "กก."),
    "ฟอง": "ฟอง",
    "ชิ้นใหญ่": "unknown",
    "มิลลิลิตร": (0.001, "ลิตร"),
    "ก้อน": "unknown",
    "ถ้วย": "unknown",
    "ช้อนโต๊ะ": (0.015, "ลิตร"),
    "ชิ้น": "unknown",
    "กร้ม": (0.001, "กก."),
    "ml": (0.001, "ลิตร"),
    "kg": (1, "กก."),
    "กก": (1, "กก."),
    "ขีด": (0.1, "กก."),
    "กิโลกรัม": (1, "กก."),
    "กรม": (0.001, "กก."),
    "ท่อน": "unknown",
    "โครง": "unknown",
    "ใบ": "ฟอง",
    "แพ็ค": "unknown",
    "mg": (0.000001, "กก."),
    "ก$": (0.001, "กก."),
    "เส้น": "unknown",
    "น่อง": "unknown",
    "ปีก": "unknown",
    "ขา": "unknown",
    "วง": "unknown",
    "ตัว": "unknown",
    "ขึด": "unknown",
    "ซอง": "unknown",
    "อก": "unknown",
    "จาน": "unknown",
    "โล": (1, "กก."),
    "แท่ง": "unknown",
    "บาท": "baht",
    "ไม้": "unknown",
    "ดุ้นสั้น": "unknown",
    "ตีน": "unknown",
    "Kg": (1, "กก."),
    "ทัพพี": "unknown",
    "ฟอว": "unit",
    "ถาด": "unknown",
    "ช้ินโต๊ะ": (0.015, "กก."),
    "ห่อ": "unknown",
    "ลูก": "unit",
    "KG": (1., "กก."),
    "แผ่น": "unknown",
    "G": (0.001, "กก."),
    "ตามสะดวก": "unknown",
    "ชต": (0.015, "กก."),
    "pack": "unknown",
    "พวง": "unknown",
    "ถุง": "unknown",
    "pcs": "ฟอง",
    "สไลด์": "unknown",
    "โคร": "unknown",
    "ฟ$": "ฟอง",
    "ชาม": "unknown",
    "แพค": "unknown",
    "ขีเ": (0.1, "กก."),
    "ช้อน$": "unknown",
    "อัน": "unknown",
    "แพ๊ค": "unknown",
    "ครึ่งโล": (0.5, "กก."),
    "หม้อ": "unknown",
    "กล่อง": "unknown",
    "กรีม": (0.001, "กก."),
    "กนัม": (0.001, "กก."),
    "สะโพก": "unknown",
    "ซีก": "unknown",
    "ขรด": "unknown",
    "ชีด": "unknown",
    "หยิบมือ": "unknown",
    "โคลง": "unknown",
    "มือ": "unknown",
    "ถต": (0.14, "กก."),
    "กรับ": (0.001, "กก."),
    "แ$": (0.001, "กก."),
    "กม": (0.001, "กก."),
    "ชุด": "unknown",
    "ช้อนโตณะ": (0.015, "กก."),
    "หอง": "ฟอง",
    "หาง": "unknown",
    "ช้อนโตํะ": "unknown",
    "ที่": "unknown",
}

patterns = [
    "\d+ - \d+\ *{}",
    "\d+-\d+\ *{}",
    "\d+\ *{}",
    "\d+\t*{}",
]

unit_patterns = []
for u in units:
    for p in patterns:
        unit_patterns.append(p.format(u))
        
unit_pattern = re.compile("|".join(unit_patterns))
unit_type_pattern = re.compile("|".join(list(units.keys())))

@app.route('/')
def search():
    kw = req.args.get('keyword')
    cursor = conn.cursor()
    cursor.execute(f"SELECT menu_name, ingredient_json FROM ingredients where menu_name ilike '%{kw}%';")
    result = cursor.fetchall()
    cursor.close()
    
    if len(result) == 0:
        return '{"msg": "not found."}'
    else:
        menus = {f"{i+1} {k}": json.loads(l) for i, (k, l) in enumerate(result)}
#         print(menus)
        
    queried_prices = {}
    menu_prices = {}
    show_str = {}
    ingd_prices = {}

    for m in menus:
        date_price = None
        show_str[m] = []
        for ig in menus[m]:
            _use_to_cal = False
            _ig = ig.split(" ")[0].replace("\u200b", "").replace("(", "").replace(")", "")
            if includes.search(_ig) and not excludes.search(_ig) and not excludes_units.search(menus[m][ig]):
                ingd_type = ingd_pattern.search(_ig)
                ingd_default_type = ingd_default_pattern.search(_ig)
                if ingd_type or ingd_default_type:
                    if ingd_type:
                        str_ingd_type = ingredient_translate[ingd_type.group()]
                    else:
                        str_ingd_type = default_ingredient[ingd_default_type.group()]

                    unit_type = unit_pattern.search(menus[m][ig])
                    if unit_type:
                        str_unit_type = unit_type.string
                        unit_type_p = unit_type_pattern.search(menus[m][ig]).group()
    #                     print(unit_type_p)
                        try:
                            common_unit_type = units[unit_type_p]
                        except:
                            common_unit_type = units[unit_type_p+"$"]

                        if common_unit_type == "unknown":
                            show_str[m].append(f"*{_ig} {menus[m][ig]} (unknown unit type)")
                        else:
                            _frac = frac_pattern.search(menus[m][ig])
                            _num = num_pattern.search(menus[m][ig])
                            if _frac or _num:

                                _use_to_cal = True

                                if _frac:
                                    fr = _frac.group()
                                    magn = float(fr.split("/")[0])/float(fr.split("/")[1])
                                else:
                                    magn = float(_num.group())

                                if str_ingd_type not in queried_prices:
                                    prices = pd.DataFrame(dict(table_prices.query(KeyConditionExpression=Key("type").eq(str_ingd_type) & Key("date").between("2020-07-23", "2020-08-22")).items())["Items"])
                                    prices["price"] = prices["price"].astype(float)
                                    queried_prices[str_ingd_type] = prices
                                else:
                                    prices = queried_prices[str_ingd_type]

                                if not isinstance(date_price, np.ndarray):
                                    date_price = np.zeros([prices.shape[0]])

                                if isinstance(common_unit_type, tuple):
                                    show_str[m].append(f"*{_ig} {menus[m][ig]} ({common_unit_type[1]})")
                                    magn *= common_unit_type[0]
                                    date_price += (prices["price"] * magn).values
                                else:
                                    show_str[m].append(f"*{_ig} {menus[m][ig]} ({common_unit_type})")
                                    if common_unit_type == "บาท":
                                        date_price += magn
                                    else:
                                        date_price += (prices["price"] * magn).values

            if not _use_to_cal:
                show_str[m].append(f"{_ig} {menus[m][ig]}")


    #                             if str_ingd_type == "นม":
    #                             print(prices)
    #                             print(str_ingd_type, magn, common_unit_type)

        
#         print(type(date_price))
        if isinstance(date_price, np.ndarray) and not np.sum(date_price) == 0:
            menu_prices[m] = date_price.tolist()
        else:
            menu_prices[m] = 0
        
    if len(queried_prices) == 0:
        qdate = []
    else:
        k = list(queried_prices.keys())[0]
        qdate = queried_prices[k]["date"].values.tolist()
        
    return {
        "menu_prices": menu_prices,
        "queried_prices": {k:queried_prices[k]["price"].values.tolist() for k in queried_prices},
        "date": qdate,
        "show_str": show_str,
    }

if __name__ == '__main__':
    app.debug = True
    app.run(host='0.0.0.0', port=8000)

Overwriting app.py


In [ ]:
!python3 app.py

 * Serving Flask app "app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
 * Running on http://0.0.0.0:8000/ (Press CTRL+C to quit)
 * Restarting with stat
 * Debugger is active!
 * Debugger PIN: 257-460-940
49.230.13.211 - - [26/Jul/2020 11:56:55] "GET /?keyword=วุ้นเส้นหมูกรอบคั่วพริกเกลือ HTTP/1.1" 200 -
49.230.13.211 - - [26/Jul/2020 11:57:05] "GET /?keyword=หมี่กรอบผัดซีอิ๊ว HTTP/1.1" 200 -


In [2]:
# import subprocess as sp

# server = sp.Popen("FLASK_APP=app.py flask run", shell=True)
# server

In [339]:
from flask import Flask, Response, render_template, make_response, jsonify, request as req
from boto3.dynamodb.conditions import Key, Attr
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import psycopg2
import boto3
import json
import re
import os

load_dotenv('.env')
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY = os.getenv("AWS_SECRET_ACCESS_KEY")

dynamodb = boto3.resource('dynamodb', region_name='ap-southeast-2',
  aws_access_key_id=AWS_ACCESS_KEY_ID,
  aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)

table_prices = dynamodb.Table('IngredientPrices')

conn = psycopg2.connect(
    "dbname=eltrial user=top password=1234"
)

kw = ""
cursor = conn.cursor()
cursor.execute(f"SELECT menu_name, ingredient_json FROM ingredients where menu_name ilike '%{kw}%';")
result = cursor.fetchall()
cursor.close()

In [255]:
includes = re.compile("หมู|ไก่|วัว|^ไข่$|ไข่ไก่|^นม")
excludes = re.compile("นมข้นหวาน|ผง|คนอร์|มาม่า|คาร์เนชั่น|หมูกรอบ|กระดูก|ปู|^ซอส|^เครื่องเทศ|^เต้าหู้|อินทผาลัม|ลูกชิ้น|หมูแฮม|ไข่แดงเค็ม|น้ำมัน|คอร์|ไส้กรอก|หัวใจ|กระเพาะ|ปอด|รสดี|คนอ|น้ำซุป|หมูยอ|น้ำ|ซีอิ๊ว|ชุด|^พริกไทย|ซีอิ้ว|สต๊อก|แป้ง|^พริก|เลือด|ชุป|รากผักชี|บะหมี่กึ่งสำเร็จรูป|ข้นหวาน|กากหมู|นมข้น|หมูแดง|แคปหมู|มันหมู|^ซอส|นมถั่วเหลือง|\*\*|^เหล้าจีน|อัลมอนด์|หมูหยอง|สูตร|ซุปก้อน|^เครื่อง|ส่วนผสม|แคบหมู")
excludes_units = re.compile("สำหรับ|ครึ่1/2|ปริมานตามใจชอบ")



In [256]:
menus = {k: json.loads(l) for k, l in result}

In [337]:
ingredient_kws = re.compile('pork_ขาหมู|pork_ซี่โครงหมู|pork_ตับหมู|pork_เนื้อก้อน|\
pork_เนื้อแดง|pork_เนื้อไหล่|pork_สะโพกหมู|pork_สันนอก|pork_สันใน|\
pork_สามชั้น|pork_ไส้ตันหมู|pork_ไส้ใหญ่หมู|pork_ไส้อ่อนหมู|pork_หมูบด|\
beef_ขอบกระด้ง|beef_ขี้ริ้ว|beef_เครื่องใน|beef_ดอกจอก|beef_น่อง|beef_ซี่โครง|\
beef_เนื้อแดง|beef_เนื้อปลีก|beef_เศษเนื้อ|beef_สันคอ|beef_เนื้อวัวบด|beef_เนื้อก้อน|\
beef_สันใน|beef_สันนอก|chicken_ไก่บ้าน|chicken_ไก่ทั้งตัว|chicken_ไก่ผ่าซีก|\
chicken_อกไก่|chicken_ซี่โครงไก่|chicken_ปีกบนไก่|chicken_ปีกกลางไก่|\
chicken_ปีกไก่เต็ม|chicken_น่องไก่|chicken_สะโพกไก่|chicken_เครื่องในไก่|\
chicken_ตีนไก่|chicken_เนื้อไก่บด|chicken_ไก่มีชีวิตหน้าฟาร์ม|eggs_ไข่ไก่เบอร์ 0|\
eggs_ไข่ไก่เบอร์ 1|eggs_ไข่ไก่เบอร์ 2|eggs_ไข่ไก่เบอร์ 3|eggs_ไข่ไก่เบอร์ 4|\
eggs_ไข่ไก่เบอร์ 5|eggs_ไข่ไก่ต้ม|eggs_ไข่ไก่คละหน้าฟาร์ม|นม')

ingredient_translate = {
    "หมูสับ": "pork_เนื้อก้อน",
    "เนื้อหมูบด": "pork_หมูบด",
    "หมูบด": "pork_หมูบด",
    "ไข่ไก่": "eggs_ไข่ไก่เบอร์ 2",
    "ไขไก่": "eggs_ไข่ไก่เบอร์ 2",
    "หมูสามชั้น": "pork_สามชั้น",
    "เนื้อหมู": "pork_เนื้อก้อน",
    "ปีกไก่บน": "chicken_ปีกบนไก่", 
    "นมสด": "นม",
    "สันในไก่": "chicken_อกไก่",
    "อกไก่": "chicken_อกไก่",
    "มันไก่": "chicken_น่องไก่",
    "ไก่สด": "chicken_อกไก่",
    "สะโพกไก่": "chicken_สะโพกไก่",
    "สันคอหมู": "pork_เนื้อก้อน",
    "คอหมู": "pork_เนื้อก้อน",
    "ซี่โครงหมู": "pork_ซี่โครงหมู",
    "ปีกไก่กลาง": "chicken_ปีกกลางไก่",
    "ไก่อก": "chicken_อกไก่",
    "หมูสันนอก": "pork_สันนอก",
    "ตับไก่": "chicken_เครื่องในไก่",
    "เนื้อไก่": "chicken_อกไก่",
    "ปีกไก่": "chicken_ปีกไก่เต็ม",
    "หมูส่วนสะโพก": "pork_สะโพกหมู",
    "หมูสันใน": "pork_สันใน",
    "เนื่อหมู": "pork_เนื้อก้อน",
    "เนื้อสันในวัว": "beef_สันใน",
    "เนื้อวัว": "beef_เนื้อก้อน",
    "โครงไก่": "chicken_ซี่โครงไก่",
    "สันนอกหมู": "pork_สันนอก",
    "น่องไก่": "chicken_น่องไก่",
    "ไก่บด": "chicken_เนื้อไก่บด",
    "ปีกบนไก่": "chicken_ปีกบนไก่",
    "หมูบก": "pork_หมูบด",
    "หมูสามชั้น": "pork_สามชั้น",
    "ใส้หมู": "pork_ไส้อ่อนหมู",
    "ไส้หมู": "pork_ไส้อ่อนหมู",
    "หนังไก่": "chicken_น่องไก่",
    "ไส้ใหญ่หมู": "pork_ไส้ใหญ่หมู",
    "เอ็นข้อไก่": "chicken_ตีนไก่",
    "สันในหมู": "pork_สันใน",
    "ตีนไก่": "chicken_ตีนไก่",
    "ตับหมู": "pork_ตับหมู",
    "ปีกกลางไก่": "chicken_ปีกกลางไก่",
    "ไก่สับ": "chicken_น่องไก่",
    "หนังหมู": "pork_สามชั้น",
}

default_ingredient = {
    "นม": "นม",
    "ไข่": "eggs_ไข่ไก่เบอร์ 2",
    "หมู": "pork_เนื้อก้อน",
    "ไก่": "chicken_น่องไก่",
    "วัว": "beef_เนื้อก้อน",
}

ingd_patterns = []
for u in ingredient_translate:
        ingd_patterns.append(u)
        
ingd_pattern = re.compile("|".join(ingd_patterns))

ingd_default_patterns = []
for u in default_ingredient:
        ingd_default_patterns.append(u)
        
ingd_default_pattern = re.compile("|".join(ingd_default_patterns))

units = {
    "กรัม": (0.001, "กก."),
    "g": (0.001, "กก."),
    "grams": (0.001, "กก."),
    "กระม": (0.001, "กก."),
    "กิโล": (1., "กก."),
    "ฟอง": "ฟอง",
    "ชิ้นใหญ่": "unknown",
    "มิลลิลิตร": (0.001, "ลิตร"),
    "ก้อน": "unknown",
    "ถ้วย": "unknown",
    "ช้อนโต๊ะ": (0.015, "ลิตร"),
    "ชิ้น": "unknown",
    "กร้ม": (0.001, "กก."),
    "ml": (0.001, "ลิตร"),
    "kg": (1, "กก."),
    "กก": (1, "กก."),
    "ขีด": (0.1, "กก."),
    "กิโลกรัม": (1, "กก."),
    "กรม": (0.001, "กก."),
    "ท่อน": "unknown",
    "โครง": "unknown",
    "ใบ": "ฟอง",
    "แพ็ค": "unknown",
    "mg": (0.000001, "กก."),
    "ก$": (0.001, "กก."),
    "เส้น": "unknown",
    "น่อง": "unknown",
    "ปีก": "unknown",
    "ขา": "unknown",
    "วง": "unknown",
    "ตัว": "unknown",
    "ขึด": "unknown",
    "ซอง": "unknown",
    "อก": "unknown",
    "จาน": "unknown",
    "โล": (1, "กก."),
    "แท่ง": "unknown",
    "บาท": "baht",
    "ไม้": "unknown",
    "ดุ้นสั้น": "unknown",
    "ตีน": "unknown",
    "Kg": (1, "กก."),
    "ทัพพี": "unknown",
    "ฟอว": "unit",
    "ถาด": "unknown",
    "ช้ินโต๊ะ": (0.015, "กก."),
    "ห่อ": "unknown",
    "ลูก": "unit",
    "KG": (1., "กก."),
    "แผ่น": "unknown",
    "G": (0.001, "กก."),
    "ตามสะดวก": "unknown",
    "ชต": (0.015, "กก."),
    "pack": "unknown",
    "พวง": "unknown",
    "ถุง": "unknown",
    "pcs": "ฟอง",
    "สไลด์": "unknown",
    "โคร": "unknown",
    "ฟ$": "ฟอง",
    "ชาม": "unknown",
    "แพค": "unknown",
    "ขีเ": (0.1, "กก."),
    "ช้อน$": "unknown",
    "อัน": "unknown",
    "แพ๊ค": "unknown",
    "ครึ่งโล": (0.5, "กก."),
    "หม้อ": "unknown",
    "กล่อง": "unknown",
    "กรีม": (0.001, "กก."),
    "กนัม": (0.001, "กก."),
    "สะโพก": "unknown",
    "ซีก": "unknown",
    "ขรด": "unknown",
    "ชีด": "unknown",
    "หยิบมือ": "unknown",
    "โคลง": "unknown",
    "มือ": "unknown",
    "ถต": (140, "g"),
    "กรับ": (0.001, "กก."),
    "แ$": (0.001, "กก."),
    "กม": (0.001, "กก."),
    "ชุด": "unknown",
    "ช้อนโตณะ": (0.015, "กก."),
    "หอง": "ฟอง",
    "หาง": "unknown",
    "ช้อนโตํะ": "unknown",
    "ที่": "unknown",
}

patterns = [
    "\d+ - \d+\ *{}",
    "\d+-\d+\ *{}",
    "\d+\ *{}",
    "\d+\t*{}",
]

unit_patterns = []
for u in units:
    for p in patterns:
        unit_patterns.append(p.format(u))
        
unit_pattern = re.compile("|".join(unit_patterns))
unit_type_pattern = re.compile("|".join(list(units.keys())))


In [308]:
unit_type_pattern

re.compile(r'กรัม|g|grams|กระม|กิโล|ฟอง|ชิ้นใหญ่|มิลลิลิตร|ก้อน|ถ้วย|ช้อนโต๊ะ|ชิ้น|กร้ม|ml|kg|กก|ขีด|กิโลกรัม|กรม|ท่อน|โครง|ใบ|แพ็ค|mg|ก$|เส้น|น่อง|ปีก|ขา|วง|ตัว|ขึด|ซอง|อก|จาน|โล|แท่ง|บาท|ไม้|ดุ้นสั้น|ตีน|Kg|ทัพพี|ฟอว|ถาด|ช้ินโต๊ะ|ห่อ|ลูก|KG|แผ่น|G|ตามสะดวก|ชต|pack|พวง|ถุง|pcs|สไลด์|โคร|ฟ$|ชาม|แพค|ขีเ|ช้อน$|อัน|แพ๊ค|ครึ่งโล|หม้อ|กล่อง|กรีม|กนัม|สะโพก|ซีก|ขรด|ชีด|หยิบมือ|โคลง|มือ|ถต|กรับ|แ$|กม|ชุด|ช้อนโตณะ|หอง|หาง|ช้อนโตํะ|ที่',
re.UNICODE)

In [315]:
frac_pattern = re.compile("\d+\/\d+")
num_pattern = re.compile("\d+")

In [336]:
dict(prices())

{'Items': [{'price': Decimal('182.01702163319416'),
   'date': '2020-07-23',
   'type': 'pork_สามชั้น'},
  {'price': Decimal('181.8467674206031'),
   'date': '2020-07-24',
   'type': 'pork_สามชั้น'},
  {'price': Decimal('181.994167428968'),
   'date': '2020-07-25',
   'type': 'pork_สามชั้น'},
  {'price': Decimal('182.24495456567672'),
   'date': '2020-07-26',
   'type': 'pork_สามชั้น'},
  {'price': Decimal('182.30987309635077'),
   'date': '2020-07-27',
   'type': 'pork_สามชั้น'},
  {'price': Decimal('182.3756543486512'),
   'date': '2020-07-28',
   'type': 'pork_สามชั้น'},
  {'price': Decimal('182.4579703953696'),
   'date': '2020-07-29',
   'type': 'pork_สามชั้น'},
  {'price': Decimal('182.5827306438472'),
   'date': '2020-07-30',
   'type': 'pork_สามชั้น'},
  {'price': Decimal('182.69303899636168'),
   'date': '2020-07-31',
   'type': 'pork_สามชั้น'},
  {'price': Decimal('182.7896671459574'),
   'date': '2020-08-01',
   'type': 'pork_สามชั้น'},
  {'price': Decimal('182.8903404284690

In [345]:
prices

,price,date,type
0,2.835654172231381,2020-07-23,eggs_ไข่ไก่เบอร์ 2
1,2.8416331494672877,2020-07-24,eggs_ไข่ไก่เบอร์ 2
2,2.8475087490903706,2020-07-25,eggs_ไข่ไก่เบอร์ 2
3,2.853282758518512,2020-07-26,eggs_ไข่ไก่เบอร์ 2
4,2.8589569342648127,2020-07-27,eggs_ไข่ไก่เบอร์ 2
5,2.8645330024719384,2020-07-28,eggs_ไข่ไก่เบอร์ 2
6,2.870012659437233,2020-07-29,eggs_ไข่ไก่เบอร์ 2
7,2.8753975721287466,2020-07-30,eggs_ไข่ไก่เบอร์ 2
8,2.8806893786923475,2020-07-31,eggs_ไข่ไก่เบอร์ 2
9,2.8858896889500607,2020-08-01,eggs_ไข่ไก่เบอร์ 2


In [360]:
queried_prices = {}
menu_prices = {}
show_str = {}
ingd_prices = {}

for m in menus:
    date_price = None
    show_str[m] = []
    for ig in menus[m]:
        _use_to_cal = False
        _ig = ig.split(" ")[0].replace("\u200b", "").replace("(", "").replace(")", "")
        if includes.search(_ig) and not excludes.search(_ig) and not excludes_units.search(menus[m][ig]):
            ingd_type = regex_pattern.search(_ig)
            ingd_default_type = ingd_default_pattern.search(_ig)
            if ingd_type or ingd_default_type:
                if ingd_type:
                    str_ingd_type = ingredient_translate[ingd_type.group()]
                else:
                    str_ingd_type = default_ingredient[ingd_default_type.group()]
                    
                unit_type = unit_pattern.search(menus[m][ig])
                if unit_type:
                    str_unit_type = unit_type.string
                    unit_type_p = unit_type_pattern.search(menus[m][ig]).group()
#                     print(unit_type_p)
                    try:
                        common_unit_type = units[unit_type_p]
                    except:
                        common_unit_type = units[unit_type_p+"$"]
                        
                    if common_unit_type == "unknown":
                        show_str[m].append(f"*{_ig} {menus[m][ig]} (unknown unit type)")
                    else:
                        _frac = frac_pattern.search(menus[m][ig])
                        _num = num_pattern.search(menus[m][ig])
                        if _frac or _num:
                            
                            _use_to_cal = True
                            
                            if _frac:
                                fr = _frac.group()
                                magn = float(fr.split("/")[0])/float(fr.split("/")[1])
                            else:
                                magn = float(_num.group())
                                
                            if str_ingd_type not in queried_prices:
                                prices = pd.DataFrame(dict(table_prices.query(KeyConditionExpression=Key("type").eq(str_ingd_type) & Key("date").between("2020-07-23", "2020-08-22")).items())["Items"])
                                prices["price"] = prices["price"].astype(float)
                                queried_prices[str_ingd_type] = prices
                            else:
                                prices = queried_prices[str_ingd_type]
                                
                            if not isinstance(date_price, np.ndarray):
                                date_price = np.zeros([prices.shape[0]])
                                
                            if isinstance(common_unit_type, tuple):
                                show_str[m].append(f"*{_ig} {menus[m][ig]} ({common_unit_type[1]})")
                                magn *= common_unit_type[0]
                                date_price += prices["price"] * magn
                            else:
                                show_str[m].append(f"*{_ig} {menus[m][ig]} ({common_unit_type})")
                                if common_unit_type == "บาท":
                                    date_price += magn
                                else:
                                    date_price += prices["price"] * magn
                                    
        if not _use_to_cal:
            show_str[m].append(f"{_ig} {menus[m][ig]}")
                                    
                            
#                             if str_ingd_type == "นม":
#                             print(prices)
#                             print(str_ingd_type, magn, common_unit_type)
        
    if isinstance(date_price, np.ndarray):
        menu_prices.append(date_price)
    else:
        menu_prices.append(0)
#     print(date_price)
#                         show_str[m].append(f"*{_ig} {menus[m][ig]} (unknown unit type)")
#                     print(str_ingd_type, common_unit_type)
                    
#                 show_str[m].append(f"*{_ig} {menus[m][ig]}")
                
#                 ingd_prices.append()
#         else:
# #             print(ig, includes.search(ig), excludes.search(ig))
#             show_str[m].append(f"{ig} {menus[m][ig]}")

In [361]:
show_str

{'ไข่เจียว': ['*ไข่ 2 ฟอง (ฟอง)', 'น้ำ 1 ถ้วยตวง', 'น้ำปลา นิดหน่อย'],
 'ไข่เจียวทูน่าพริกสับ': ['ปลาทูน่า 1 กระป๋อง',
  'พริกขี้หนู 6 เม็ด',
  '*ไข่ไก่ 2 ฟอง (ฟอง)',
  'น้ำปลา เล็กน้อย',
  'น้ำมันพืช พอประมาณ'],
 'วุ้นเส้นหมูกรอบคั่วพริกเกลือ': ['วุ้นเส้น 2 ห่อ\xa0\xa0',
  '*หมูสามชั้น 400 กรัม\xa0 (กก.)',
  'เกลือ 1 ช้อนโต๊ะ ',
  'น้ำส้มสายชู 2 ช้อนโต๊ะ',
  'พริกแดงจินดาสับ 1½ ช้อนโต๊ะ',
  'กระเทียมสับ 1½ ช้อนโต๊ะ',
  'เกลือ 1 ช้อนชา',
  'น้ำตาล 1 ช้อนชา\xa0',
  'ซีอิ๊วขาวต้นหอม 1 ช้อนโต๊ะ\xa0'],
 'ขนมผิง🌟': ['แป้งมัน 2 ถ้วยตวง',
  'น้ำตาลทราย 150 กรัม',
  'กะทิอบควันเทียน 1/2 ถ้วยตวง',
  'ไข่แดง 1 ฟอง',
  'น้ำเปล่า 2 ช้อนโต๊ะ'],
 'หมี่กรอบผัดซีอิ๊ว': ['บะหมี่ไข่ 2 1/2 ก้อน',
  'น้ำมันพืช พอประมาณ',
  '*ไข่ไก่ 2 ฟอง (ฟอง)',
  '*เนื้อหมู 1 ชิ้นใหญ่ (unknown unit type)',
  'เนื้อหมู 1 ชิ้นใหญ่',
  'คะน้า 3 ต้น',
  'น้ำตาลทราย ตามชอบ',
  'ผงปรุงรส ตามชอบ',
  'ซีอิ๊วดำ เล็กน้อย',
  'พริกไทยขาวป่น ตามชอบ'],
 'ไก่ทอดเกาหลีซอสส้ม': ['แป้งมันสำปะหลัง 2 ช้อนโต๊ะ',
  'น้ำสะอาด 30 มิลลิลิตร',
 